In [1]:
import os
import pandas as pd
import numpy as np

### PARTE 1: ETL das bases que serão usadas

In [2]:
import pandas as pd

# Carregar a base de dados
caminho = r"C:\Users\anabe\Documents\FIAP\Ano 2\0. Challenge\Challenge_TOTVS_2025_MassaDados_v1\RAW\dados_clientes.csv"
df = pd.read_csv(caminho, sep=";", decimal=",")

# Converter a coluna de data para o formato datetime
df["DT_ASSINATURA_CONTRATO"] = pd.to_datetime(df["DT_ASSINATURA_CONTRATO"], errors="coerce")

# Filtrar apenas contratos a partir de 2015
# Justificativa: contratos mais antigos (ex.: 1993) apresentavam concentrações atípicas de receita,
# o que poderia distorcer a análise
df = df[df["DT_ASSINATURA_CONTRATO"].dt.year >= 2015]

# Selecionar apenas as colunas de interesse para a análise
colunas_interesse = [
    "CD_CLIENTE",
    "DS_PROD",
    "DS_LIN_REC",
    "DS_SEGMENTO",
    "FAT_FAIXA",
    "SITUACAO_CONTRATO",
    "VL_TOTAL_CONTRATO",
    "DT_ASSINATURA_CONTRATO"
]
df = df[colunas_interesse].copy()

# Padronizar textos (remoção de espaços extras e transformação para maiúsculas)
colunas_texto = ["CD_CLIENTE", "DS_LIN_REC", "DS_SEGMENTO", "FAT_FAIXA", "SITUACAO_CONTRATO", "VL_TOTAL_CONTRATO", "DT_ASSINATURA_CONTRATO","DS_PROD"]
for col in colunas_texto:
    df[col] = df[col].astype(str).str.strip().str.upper()

# Agrupar valores da coluna de produtos DS_LIN_REC em categorias mais gerais
def agrupar_ds_lin_rec(valor):
    if pd.isna(valor):
        return "OUTROS"
    v = valor.upper()
    if v.startswith("SAAS"):
        return "SAAS"
    elif v.startswith("CDU"):
        return "CDU"
    elif v.startswith("SMS"):
        return "SMS"
    elif any(x in v for x in ["CONSULTORIA", "SERVICO", "SERVICE", "BPO", "FABRICA"]):
        return "CONSULTORIA & SERVIÇOS"
    elif "CLOUD" in v or "CLOUDFICACAO" in v:
        return "CLOUD"
    else:
        return "OUTROS"

df["DS_LIN_REC_AGRUPADA"] = df["DS_LIN_REC"].apply(agrupar_ds_lin_rec)

# Garantir que a coluna de valores de contrato seja numérica
df["VL_TOTAL_CONTRATO"] = pd.to_numeric(df["VL_TOTAL_CONTRATO"], errors="coerce")

# A partir da analise da distribuição dos clientes por faixas de receita, escolhemos reagrupar dessa forma abaixo
# Criar mapeamento manual das faixas
mapa_fat_faixa = {
    "SEM INFORMACOES DE FATURAMENTO": "SEM_INFO",
    "FAIXA 00 - ATE 4,5 M": "ATE 15M",
    "FAIXA 01 - DE 4,5 M ATE 7,5 M": "ATE 15M",
    "FAIXA 02 - DE 7,5 M ATE 15 M": "ATE 15M",
    "FAIXA 03 - DE 15 M ATE 25 M": "15M-50M",
    "FAIXA 04 - DE 25 M ATE 35 M": "15M-50M",
    "FAIXA 05 - DE 35 M ATE 50 M": "15M-50M",
    "FAIXA 06 - DE 50 M ATE 75 M": "50M-150M",
    "FAIXA 07 - DE 75 M ATE 150 M": "50M-150M",
    "FAIXA 08 - DE 150 M ATE 300 M": "ACIMA 150M",
    "FAIXA 09 - DE 300 M ATE 500 M": "ACIMA 150M",
    "FAIXA 10 - DE 500 M ATE 850 M": "ACIMA 150M",
    "FAIXA 11 - ACIMA DE 850 M": "ACIMA 150M"
}

# Padronizar e mapear
df["FAT_FAIXA"] = df["FAT_FAIXA"].str.upper().str.strip()
df["FAT_FAIXA_AGRUPADA"] = df["FAT_FAIXA"].map(mapa_fat_faixa).fillna("SEM_INFO")
df["FAT_FAIXA_AGRUPADA"] = df["FAT_FAIXA_AGRUPADA"].astype("category")

# Conferência final: exibir as primeiras linhas do dataframe transformado
print(df.head())


   CD_CLIENTE                      DS_PROD         DS_LIN_REC  \
0       99958          SMS FULL TOTVS TRAD  SMS TOTVS SERIE T   
3       99958          CDU FULL TOTVS TRAD  CDU TOTVS SERIE T   
5       99069    TOTVS RH ATS PACK 3 VAGAS       SAAS SERIE T   
18     T00082  PACK TOTVS AUDITOR NFE 1000       SAAS SERIE T   
35     T00018   SMS LGT G. CAP HUMANO TRAD  SMS TOTVS SERIE T   

              DS_SEGMENTO                       FAT_FAIXA SITUACAO_CONTRATO  \
0                SERVICOS   FAIXA 09 - DE 300 M ATE 500 M          GRATUITO   
3                SERVICOS   FAIXA 09 - DE 300 M ATE 500 M          GRATUITO   
5                  VAREJO  SEM INFORMAÇÕES DE FATURAMENTO          GRATUITO   
18  CONSTRUCAO E PROJETOS     FAIXA 04 - DE 25 M ATE 35 M             ATIVO   
35               SERVICOS    FAIXA 02 - DE 7,5 M ATE 15 M             ATIVO   

    VL_TOTAL_CONTRATO DT_ASSINATURA_CONTRATO DS_LIN_REC_AGRUPADA  \
0        1.633817e+06             2016-07-04                 SMS  

In [3]:
import pandas as pd

### Integração com a base clientes_desde.csv
# Caminho do arquivo
caminho_desde = r'C:/Users/anabe/Documents/FIAP/Ano 2/0. Challenge/Challenge_TOTVS_2025_MassaDados_v1/RAW/clientes_desde.csv'

# Leitura correta com tratamento de BOM e separador
df_desde = pd.read_csv(caminho_desde, sep=';', encoding='utf-8-sig')

# Converter a coluna de datas
df_desde['CLIENTE_DESDE'] = pd.to_datetime(df_desde['CLIENTE_DESDE'], errors='coerce')

df_desde.rename(columns={'CLIENTE': 'CD_CLIENTE'}, inplace=True)

df_desde["ANO_INICIO"] = df_desde["CLIENTE_DESDE"].dt.year

# 4. Tempo de relacionamento em anos
hoje = pd.to_datetime("today")
df_desde["TEMPO_CLIENTE_ANOS"] = (
    (hoje - df_desde["CLIENTE_DESDE"]).dt.days // 365
)


In [4]:
# Vamos agrupar os tempos de relacionamento dos clientes conforme as faixas "Recente (0-3 anos)", "Estabelecido (3-10 anos)", "Madura (>10 anos)"

bins = [0, 3, 10, np.inf]
labels = ["Recente (0-3)", "Estabelecido (3-10)", "Madura (>10)"]

df_desde["TEMPO_CLIENTE_FAIXA"] = pd.cut(
    df_desde["TEMPO_CLIENTE_ANOS"],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
).astype("category")


df_merged = pd.merge(
    df, 
    df_desde, 
    on="CD_CLIENTE",     # chave de junção
    how="left"           # mantém todos os clientes do df_agrupado
)

df_merged.head()



,CD_CLIENTE,DS_PROD,DS_LIN_REC,DS_SEGMENTO,FAT_FAIXA,SITUACAO_CONTRATO,VL_TOTAL_CONTRATO,DT_ASSINATURA_CONTRATO,DS_LIN_REC_AGRUPADA,FAT_FAIXA_AGRUPADA,CLIENTE_DESDE,ANO_INICIO,TEMPO_CLIENTE_ANOS,TEMPO_CLIENTE_FAIXA
0,99958,SMS FULL TOTVS TRAD,SMS TOTVS SERIE T,SERVICOS,FAIXA 09 - DE 300 M ATE 500 M,GRATUITO,1.633817e+06,2016-07-04,SMS,ACIMA 150M,2016-04-07,2016,9,Estabelecido (3-10)
1,99958,CDU FULL TOTVS TRAD,CDU TOTVS SERIE T,SERVICOS,FAIXA 09 - DE 300 M ATE 500 M,GRATUITO,4.203437e+01,2016-07-04,CDU,ACIMA 150M,2016-04-07,2016,9,Estabelecido (3-10)
2,99069,TOTVS RH ATS PACK 3 VAGAS,SAAS SERIE T,VAREJO,SEM INFORMAÇÕES DE FATURAMENTO,GRATUITO,4.203857e-03,2023-11-01,SAAS,SEM_INFO,2017-08-31,2017,8,Estabelecido (3-10)
3,T00082,PACK TOTVS AUDITOR NFE 1000,SAAS SERIE T,CONSTRUCAO E PROJETOS,FAIXA 04 - DE 25 M ATE 35 M,ATIVO,5.108359e+02,2018-08-05,SAAS,15M-50M,2010-06-07,2010,15,Madura (>10)
4,T00018,SMS LGT G. CAP HUMANO TRAD,SMS TOTVS SERIE T,SERVICOS,"FAIXA 02 - DE 7,5 M ATE 15 M",ATIVO,2.030631e+02,2015-02-09,SMS,ATE 15M,2002-01-01,2002,23,Madura (>10)


In [5]:
# chegamos na base final, antes do merge com as infos de clustering

print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39056 entries, 0 to 39055
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   CD_CLIENTE              39056 non-null  object        
 1   DS_PROD                 39056 non-null  object        
 2   DS_LIN_REC              39056 non-null  object        
 3   DS_SEGMENTO             39056 non-null  object        
 4   FAT_FAIXA               39056 non-null  object        
 5   SITUACAO_CONTRATO       39056 non-null  object        
 6   VL_TOTAL_CONTRATO       39056 non-null  float64       
 7   DT_ASSINATURA_CONTRATO  39056 non-null  object        
 8   DS_LIN_REC_AGRUPADA     39056 non-null  object        
 9   FAT_FAIXA_AGRUPADA      39056 non-null  category      
 10  CLIENTE_DESDE           39056 non-null  datetime64[ns]
 11  ANO_INICIO              39056 non-null  int32         
 12  TEMPO_CLIENTE_ANOS      39056 non-null  int64 

### Merge com dados de clusters

In [9]:
import pandas as pd
from pathlib import Path

# caminhos

# 1) ler de-para (simples)
p_depara = r"C:\Users\anabe\Documents\FIAP\Ano 2\0. Challenge\Challenge_TOTVS_2025_MassaDados_v1\Clusters Results\df_depara_clientid_clusters.csv"
df_depara = pd.read_csv(p_depara)
df_depara["Cluster"] = pd.to_numeric(df_depara["Cluster"], errors="raise")

# 2) ler nomes com parser tolerante
p_nomes = r"C:\Users\anabe\Documents\FIAP\Ano 2\0. Challenge\Challenge_TOTVS_2025_MassaDados_v1\Clusters Results\Tabela_de_Clusters_Nomeados.csv"

df_nome_cluster = pd.read_csv(p_nomes, engine="python")
df_nome_cluster = df_nome_cluster["Cluster ID"].str.split(",", n=2, expand=True)
df_nome_cluster.columns = ["Cluster ID", "Título do Cluster", "Descrição do Cluster"]

# 2c) converter Cluster ID para int
df_nome_cluster["Cluster ID"] = pd.to_numeric(df_nome_cluster["Cluster ID"], errors="raise")

# 3) merges
df_out = (
    df_merged
      .merge(df_depara, on="CD_CLIENTE", how="left")
      .merge(df_nome_cluster, left_on="Cluster", right_on="Cluster ID", how="left")
)

# 4) organizar colunas e exportar
cols_first = ["CD_CLIENTE", "Cluster", "Título do Cluster", "Descrição do Cluster"]
cols_rest  = [c for c in df_out.columns if c not in cols_first]
df_out = df_out[cols_first + cols_rest]

df_out.to_csv(p_out, index=False, encoding="utf-8-sig")
print("OK! Linhas:", len(df_out), "| salvo em:", p_out)


OK! Linhas: 39056 | salvo em: C:\Users\anabe\Documents\FIAP\Ano 2\0. Challenge\Challenge_TOTVS_2025_MassaDados_v1\Clusters Results\df_merged_com_clusters_nomeados.csv
